In [36]:
#import useful packages 

import numpy as np
import scipy.interpolate as sp_intpl
import pandas as pd
import scipy.integrate as sp_int

In [58]:
#reads the data from spreadsheet

stream_data = pd.read_csv('../EEB/stream_data_UBC.csv')
stream_data

,Trasect,Right Bank,Right Center,Center,Left Center,Left Bank,Width
0,0,0,19,26,42,0,7.50
1,5,0,13,16,30,0,7.13
2,10,0,8,7,12,0,5.83
3,15,0,35,13,16,0,6.97
4,20,0,9,29,42,0,6.63
5,25,0,15,11,25,0,4.52
6,30,0,18,7,12,0,6.45
7,35,0,12,14,18,0,7.97
8,40,0,16,5,26,0,11.89
9,45,0,0,32,48,0,11.20


In [32]:
#data in a format to make the interpolant 

s = np.float64(stream_data['Trasect'])
w1 = np.array([-1, -0.5, 0, 0.5, 1])
w = np.reshape(np.float64(stream_data['Width']) / 2, (len(s), 1))
depths = np.float64(stream_data[['Right Bank', 'Right Center', 'Center', 'Left Center', 'Left Bank']])



In [40]:
#make intepolant 
I = sp_intpl.RectBivariateSpline(s, w1, w * depths)

#do integral 
V = sp_int.nquad(I, [[0, 100], [-1, 1]])

In [147]:
def get_Volume(stream_data):
    
    s = np.float64(stream_data['Trasect'])
    w1 = np.array([-1, -0.5, 0, 0.5, 1])
    w = np.reshape(np.float64(stream_data['Width']) / 2, (len(s), 1))
    depths = np.float64(stream_data[['Right Bank', 'Right Center', 'Center', 'Left Center', 'Left Bank']])

    #make intepolant 
    I = sp_intpl.RectBivariateSpline(s, w1, w * depths, kx = 3, ky = 3)

    #do integral 
    V = sp_int.nquad(I, [[0, 100], [-1, 1]])

    return V

    

In [148]:
get_Volume(stream_data)[0]

/Users/ethan/opt/anaconda3/envs/chianti/lib/python3.8/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


11796.901373474733

In [149]:
#test on fake data 

cylinder = pd.DataFrame(data = {'Trasect' : np.linspace(0, 100, 21), 'Right Bank' : np.zeros(21), 'Right Center' : np.sqrt(5**2 - (5/2)**2) * np.ones(21), 'Center' : 5.0*np.ones(21), 'Left Center': np.sqrt(5**2 - (5/2)**2) * np.ones(21), 'Left Bank':np.zeros(21), 'Width' : 10.0*np.ones(21)})

In [150]:
cylinder

,Trasect,Right Bank,Right Center,Center,Left Center,Left Bank,Width
0,0.0,0.0,4.330127,5.0,4.330127,0.0,10.0
1,5.0,0.0,4.330127,5.0,4.330127,0.0,10.0
2,10.0,0.0,4.330127,5.0,4.330127,0.0,10.0
3,15.0,0.0,4.330127,5.0,4.330127,0.0,10.0
4,20.0,0.0,4.330127,5.0,4.330127,0.0,10.0
5,25.0,0.0,4.330127,5.0,4.330127,0.0,10.0
6,30.0,0.0,4.330127,5.0,4.330127,0.0,10.0
7,35.0,0.0,4.330127,5.0,4.330127,0.0,10.0
8,40.0,0.0,4.330127,5.0,4.330127,0.0,10.0
9,45.0,0.0,4.330127,5.0,4.330127,0.0,10.0


In [151]:
#analytical volume

aV = 100 * np.pi * 5**2 / 2
print(aV)

get_Volume(cylinder)

3926.9908169872415


(3720.084679281463, 4.1302072872895224e-11)

In [152]:
#check if difference is due to lack of points in interpolant

s_c = np.linspace(0, 100, 100)
w1_c = np.linspace(-1, 1, 150)
w_c = np.reshape(5 * np.ones(len(s_c)), (len(s_c), 1)) 
ss_c, w1w1_c = np.meshgrid(s_c, w1_c, indexing = 'ij')

def get_zc(x):

    return np.sqrt(5**2 - x**2)

zz_c = w_c * get_zc(5 * w1w1_c)

I_c = sp_intpl.RectBivariateSpline(s_c, w1_c, zz_c, kx = 3, ky = 3)

V_c = sp_int.nquad(I_c, [[0, 100], [-1, 1]])



In [153]:
V_c

(3926.0742312744806, 5.1259618857072147e-05)

In [101]:
w1_c

array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ])